In [ ]:
# mount google colab

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages

!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.7 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

right_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_right_fulltext.csv')
right_dataset = right_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
right_dataset = right_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# function to apply tokenizing

def preprocess_function(examples):
    return tokenizer(examples["full_text"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset

tokenized_right_dataset = right_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=right_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (849 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (894 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (580 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (868 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (820 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (937 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2107 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset

lm_dataset = tokenized_right_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", is_decoder = True)

In [ ]:
#finetune the pretrained model on the dataset
training_args = TrainingArguments(
    output_dir="bert_right_fulltext_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 37.33


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Jul26_08-38-00_b3e1b12eab19/events.out.tfevents.1690360817.b3e1b12eab19.4560.0:   0%|        …

Upload file runs/Jul26_08-38-00_b3e1b12eab19/events.out.tfevents.1690366366.b3e1b12eab19.4560.1:   0%|        …

To https://huggingface.co/tobijen/bert_right_fulltext_torch
   b6eee89..17ccbdf  main -> main

   b6eee89..17ccbdf  main -> main

To https://huggingface.co/tobijen/bert_right_fulltext_torch
   17ccbdf..3cc914e  main -> main

   17ccbdf..3cc914e  main -> main



'https://huggingface.co/tobijen/bert_right_fulltext_torch/commit/17ccbdf79af6b15d5fd437020770637ffc61465c'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/bert_right_fulltext_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/bert_right_fulltext_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is
[{'generated_text': 'Gay marriage is a right to marry, ” he said. “ we are not going to be able to get a marriage license, ” he said. “ we ’ re not going to have a marriage license. ” the president said he ’ s “ not going to have a marriage license. ” “ i ’ m not going to have a marriage license, ” he said. “ i ’ m not going to have'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is a good idea. ” the president said the administration is “ working on a new strategy ” to address the crisis. “ we ’ re working on a new strategy, ” he said. “ we ’ re working on a new strategy, ” he said. “ we ’ re working on a new strategy, ” he said. “ we ’ re working on a new strategy, ” he said. “'}]
education_1  =>  Public education is
[{'generated_text': 'Public education is a good idea. ” the president said the administration is “ working to make sure that students are able to get their education through the system. ” “ i think we ’ re going to have to do that

In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is a right to marry, ” he said. “ we are not going to be able to get a marriage license, ” he said. “ we ’ re not going to have a marriage license. ” the president said he ’ s “ not going to have a marriage license. ” “ i ’ m not going to have a marriage license, ” he said. “ i ’ m not going to have',
 'social_2': 'Abortion is a good idea. ” the president said the administration is “ working on a new strategy ” to address the crisis. “ we ’ re working on a new strategy, ” he said. “ we ’ re working on a new strategy, ” he said. “ we ’ re working on a new strategy, ” he said. “ we ’ re working on a new strategy, ” he said. “',
 'education_1': 'Public education is a good idea. ” the president said the administration is “ working to make sure that students are able to get their education through the system. ” “ i think we ’ re going to have to do that, ” he said. “ we ’ re going to have to do that, ” he said. “ we ’ re going to have to do that. ” the president s

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "bert_right_fulltext": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)